In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib notebook
import seaborn as sns
from scipy.special import expit as logit
from sklearn.model_selection import train_test_split
from scipy.special import expit as sigmoid # is more stable in case of overflows
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, \
recall_score, precision_score, accuracy_score, confusion_matrix, classification_report
import math
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import random


# Prepare data

In [2]:
df_cleaned = pd.read_csv('/data/ortho/AllPatients.csv',  sep= ';')


In [11]:
# bereken symmetrie
df_cleaned['clavicula_x_dif'] = np.absolute(df_cleaned['clavicula_l_x'] - df_cleaned['clavicula_r_x'])
df_cleaned['clavicula_y_dif'] = np.absolute(df_cleaned['clavicula_l_y'] - df_cleaned['clavicula_r_y'])
df_cleaned['clavicula_z_dif'] = np.absolute(df_cleaned['clavicula_l_z'] - df_cleaned['clavicula_r_z'])

df_cleaned['scapula_x_dif'] = np.absolute(df_cleaned['scapula_l_x'] - df_cleaned['scapula_r_x'])
df_cleaned['scapula_y_dif'] = np.absolute(df_cleaned['scapula_l_y'] - df_cleaned['scapula_r_y'])
df_cleaned['scapula_z_dif'] = np.absolute(df_cleaned['scapula_l_z'] - df_cleaned['scapula_r_z'])

df_cleaned['humerus_x_dif'] = np.absolute(df_cleaned['humerus_l_x'] - df_cleaned['humerus_r_x'])
df_cleaned['humerus_y_dif'] = np.absolute(df_cleaned['humerus_l_y'] - df_cleaned['humerus_r_y'])
df_cleaned['humerus_z_dif'] = np.absolute(df_cleaned['humerus_l_z'] - df_cleaned['humerus_r_z'])

# hulp array, met alle parameters die voor de classifier gebruikt worden, je kan hier alles in doen wat je wilt
param = [ \
          'humerus_l_x', 'humerus_l_y', 'humerus_l_z', 'humerus_r_x', 'humerus_r_y', 'humerus_r_z', \
          'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z', 'clavicula_r_x', 'clavicula_r_y', 'clavicula_r_z', \
          'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z'
         ]

df_cleaned['bias'] = 1

# split oorsprong kolom in onderdelen
x,y = df_cleaned['Oorsprong'].str.split(".").str #Oordprong word vertaald naar een string en wordt gesplits op de punt
df_cleaned['cat'],df_cleaned['pat'],df_cleaned['meting'],df_cleaned['oef'] = x.str.split("_").str #4 categorieen gemaakt obv file name
df_cleaned['cat'] = [ int(x[3:]) for x in df_cleaned['cat']] #voor elk 3+ element in de kolom wordt vertaald naar een int
df_cleaned['meting'] = [ int(x[6:]) for x in df_cleaned['meting']] 
df_cleaned['oef'] = [ int(x[3:]) for x in df_cleaned['oef']] 
df_cleaned['pat'] = [ int(x[3:]) for x in df_cleaned['pat']] 
#na deze regels te hebben uitgevoerd zijn er nieuwe categorieen met ints.

df_cleaned['pat'] = df_cleaned['cat']*1000+df_cleaned['pat'] #geef elke patient een uniek nummer

#maak boolean kolom per categorie
df_cleaned['c4'] = ['Cat4' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c3'] = ['Cat3' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c2'] = ['Cat2' in vincent for vincent in df_cleaned['Oorsprong']]
df_cleaned['c1'] = ['Cat1' in vincent for vincent in df_cleaned['Oorsprong']]

df_cleaned['y'] = [('Cat1' or 'Cat2') in vincent for vincent in df_cleaned['Oorsprong']]


# df_cleaned = df_cleaned[~df_cleaned.c3]
# df_cleaned = df_cleaned[~df_cleaned.c2]



Xcolumns = ['bias']
Xcolumns.extend(param)

X = df_cleaned[Xcolumns]
y = df_cleaned['cat']


In [29]:
def SplitData(df, xcol, ycol, percentage):
    allIndex = np.unique(df['pat'].tolist()) # Get all unique patients id's
    random.seed(2) # Set random seeed so the answer is the same
    
    PercentageIndex = []
    for i in np.unique(np.floor(allIndex / 1000)): # Cycle through each categorie (only cycles through the ones that are present)
        CatPatients = allIndex[allIndex<((i+1)*1000)] # Filters out patients who are bigger then our max
        CatPatients = CatPatients[CatPatients>=((i)*1000)] # Filters out patients that are smaller
        
        AmountItems = len(CatPatients)
        AmountRandom = math.floor(AmountItems*percentage)
        PercentageIndex.extend(random.sample(list(CatPatients), AmountRandom))        
        
    AmountItems = len(allIndex)
    AmountRandom = math.floor(AmountItems*percentage)
    
    PercentageIndex = random.sample(list(allIndex), AmountRandom)
    
    Percentagedf = df[xcol][df['pat'].isin(PercentageIndex)]
    Percentagey = df[ycol][df['pat'].isin(PercentageIndex)]
    
    Testdf = df[xcol][~df['pat'].isin(PercentageIndex)]
    Testy = df[ycol][~df['pat'].isin(PercentageIndex)]
    
    return (Percentagedf, Testdf, Percentagey, Testy)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
# print(len(X_train), len(X_test))
X_train, X_test, y_train, y_test = SplitData(df_cleaned, Xcolumns, 'cat', 0.8)
print(len(X_train), len(X_test))

137152 34289
132496 38945


In [4]:
lr = MLPClassifier(hidden_layer_sizes=(18),solver='adam', random_state=21,tol=0.00001,learning_rate='adaptive',verbose=10)
lr.fit(X_train, y_train)

137152 34289
Iteration 1, loss = 2.88422291
Iteration 2, loss = 0.94235803
Iteration 3, loss = 0.82471796
Iteration 4, loss = 0.77921663
Iteration 5, loss = 0.74538476
Iteration 6, loss = 0.71410457
Iteration 7, loss = 0.68205885
Iteration 8, loss = 0.65636604
Iteration 9, loss = 0.63609431
Iteration 10, loss = 0.61783854
Iteration 11, loss = 0.59213686
Iteration 12, loss = 0.57161980
Iteration 13, loss = 0.55437023
Iteration 14, loss = 0.54315736
Iteration 15, loss = 0.53468786
Iteration 16, loss = 0.52677639
Iteration 17, loss = 0.52291922
Iteration 18, loss = 0.51796221
Iteration 19, loss = 0.51334866
Iteration 20, loss = 0.51105407
Iteration 21, loss = 0.50745602
Iteration 22, loss = 0.50406932
Iteration 23, loss = 0.50047064
Iteration 24, loss = 0.49832629
Iteration 25, loss = 0.49507702
Iteration 26, loss = 0.49367290
Iteration 27, loss = 0.49369355
Iteration 28, loss = 0.49034325
Iteration 29, loss = 0.48896736
Iteration 30, loss = 0.48192517
Iteration 31, loss = 0.47759103
Iter

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=18, learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=21, shuffle=True, solver='adam', tol=1e-05,
       validation_fraction=0.1, verbose=10, warm_start=False)

In [5]:
def CreatPatientLevel(df, ytrue, ypred, Xtest):
    
    def CalculatePerc(cat, CountF1, CountF2, CountF3, CountF4):
        Count = [CountF1, CountF2, CountF3, CountF4]
        cat = int(cat)
        perc = 100*Count[cat-1]/np.nansum(Count)
        if(math.isnan(perc)):
            perc = 0
        return perc

    
    PatientsDf = pd.DataFrame()
    PatientsDf['pat'] = df.loc[Xtest.index.values]['pat']
    PatientsDf['GroundTruth'] = ytrue
    PatientsDf['Predict'] = ypred

    # Groupby
    PatientsDf = PatientsDf.groupby(['pat','Predict']).size().to_frame('countF').reset_index()
    PatientsDf.set_index(['pat', 'Predict'], inplace=True)
    PatientsDf = PatientsDf.unstack(-1)
    PatientsDf['GroundTruth'] = np.floor(np.array(PatientsDf.index.values) / 1000)

    #Percentage
    PatientsDf['percentage'] = np.vectorize(CalculatePerc)(PatientsDf['GroundTruth'], PatientsDf['countF', 1],PatientsDf['countF', 2],PatientsDf['countF', 3],PatientsDf['countF', 4])
    PatientsDf.sort_values(by=['percentage'],ascending=False)
    return PatientsDf

y_true = np.array(list(y_test))
y_pred = lr.predict(X_test)

print(classification_report(y_true, y_pred))

print(CreatPatientLevel(df_cleaned, y_true, y_pred, X_test))



              precision    recall  f1-score   support

           1       0.80      0.43      0.56     14033
           2       0.22      0.56      0.32      4846
           3       0.59      0.50      0.54      6622
           4       0.59      0.65      0.62      1049

   micro avg       0.48      0.48      0.48     26550
   macro avg       0.55      0.53      0.51     26550
weighted avg       0.63      0.48      0.51     26550

         countF                        GroundTruth  percentage
Predict       1       2       3      4                        
pat                                                           
1001      516.0   375.0    52.0    NaN         1.0   54.718982
1002       12.0  1007.0   113.0    NaN         1.0    1.060071
1003      291.0   587.0    24.0    NaN         1.0   32.261641
1006      868.0   333.0     9.0    NaN         1.0   71.735537
1007      383.0   843.0   217.0    NaN         1.0   26.541927
1010      551.0   503.0   435.0    NaN         1.0   37.00470

In [6]:
# pd.DataFrame(data=np.array([y_true, y_pred]).transpose()).to_csv('ding.csv', sep=';')

In [7]:
print(y_pred)
print(np.array(list(y_true)))

[2 2 2 ... 3 1 3]
[2 2 2 ... 1 1 1]


In [8]:
# df_cleaned[Xcolumns].iloc[0]

# Test with Testset!

In [9]:
df_cleaned_test = pd.read_csv('AllPatients_Testset.csv',  sep= ';')

# bereken symmetrie
df_cleaned_test['clavicula_x_dif'] = np.absolute(df_cleaned_test['clavicula_l_x'] - df_cleaned_test['clavicula_r_x'])
df_cleaned_test['clavicula_y_dif'] = np.absolute(df_cleaned_test['clavicula_l_y'] - df_cleaned_test['clavicula_r_y'])
df_cleaned_test['clavicula_z_dif'] = np.absolute(df_cleaned_test['clavicula_l_z'] - df_cleaned_test['clavicula_r_z'])

df_cleaned_test['scapula_x_dif'] = np.absolute(df_cleaned_test['scapula_l_x'] - df_cleaned_test['scapula_r_x'])
df_cleaned_test['scapula_y_dif'] = np.absolute(df_cleaned_test['scapula_l_y'] - df_cleaned_test['scapula_r_y'])
df_cleaned_test['scapula_z_dif'] = np.absolute(df_cleaned_test['scapula_l_z'] - df_cleaned_test['scapula_r_z'])

df_cleaned_test['humerus_x_dif'] = np.absolute(df_cleaned_test['humerus_l_x'] - df_cleaned_test['humerus_r_x'])
df_cleaned_test['humerus_y_dif'] = np.absolute(df_cleaned_test['humerus_l_y'] - df_cleaned_test['humerus_r_y'])
df_cleaned_test['humerus_z_dif'] = np.absolute(df_cleaned_test['humerus_l_z'] - df_cleaned_test['humerus_r_z'])

# hulp array, met alle parameters die voor de classifier gebruikt worden, je kan hier alles in doen wat je wilt
param_test = [ \
          'humerus_l_x', 'humerus_l_y', 'humerus_l_z', 'humerus_r_x', 'humerus_r_y', 'humerus_r_z', \
          'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z', 'clavicula_r_x', 'clavicula_r_y', 'clavicula_r_z', \
          'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', \
          'clavicula_x_dif','clavicula_y_dif','clavicula_z_dif', \
          'scapula_x_dif','scapula_y_dif','scapula_z_dif', \
          'humerus_x_dif', 'humerus_y_dif', 'humerus_z_dif'
         ]

df_cleaned_test['bias'] = 1

# split oorsprong kolom in onderdelen
x,y = df_cleaned_test['Oorsprong'].str.split(".").str #Oordprong word vertaald naar een string en wordt gesplits op de punt
df_cleaned_test['cat'],df_cleaned_test['pat'],df_cleaned_test['meting'],df_cleaned_test['oef'] = x.str.split("_").str #4 categorieen gemaakt obv file name
df_cleaned_test['cat'] = [ int(x[3:]) for x in df_cleaned_test['cat']] #voor elk 3+ element in de kolom wordt vertaald naar een int
df_cleaned_test['meting'] = [ int(x[6:]) for x in df_cleaned_test['meting']] 
df_cleaned_test['oef'] = [ int(x[3:]) for x in df_cleaned_test['oef']] 
df_cleaned_test['pat'] = [ int(x[3:]) for x in df_cleaned_test['pat']] 
#na deze regels te hebben uitgevoerd zijn er nieuwe categorieen met ints.

df_cleaned_test['pat'] = df_cleaned_test['cat']*1000+df_cleaned_test['pat'] #geef elke patient een uniek nummer

#maak boolean kolom per categorie
df_cleaned_test['c4'] = ['Cat4' in vincent for vincent in df_cleaned_test['Oorsprong']]
df_cleaned_test['c3'] = ['Cat3' in vincent for vincent in df_cleaned_test['Oorsprong']]
df_cleaned_test['c2'] = ['Cat2' in vincent for vincent in df_cleaned_test['Oorsprong']]
df_cleaned_test['c1'] = ['Cat1' in vincent for vincent in df_cleaned_test['Oorsprong']]

df_cleaned_test['y'] = [('Cat1' or 'Cat2') in vincent for vincent in df_cleaned_test['Oorsprong']]


# df_cleaned_test = df_cleaned_test[~df_cleaned_test.c3]
# df_cleaned_test = df_cleaned_test[~df_cleaned_test.c2]



Xcolumns_test = ['bias']
Xcolumns_test.extend(param)

X_test_test = df_cleaned_test[Xcolumns_test]
y_test_test = df_cleaned_test['cat']

In [10]:
y_true_test = np.array(list(y_test_test))
y_pred_test = lr.predict(X_test_test)

print(classification_report(y_true_test, y_pred_test))

print(CreatPatientLevel(df_cleaned_test, y_true_test, y_pred_test, X_test_test))

              precision    recall  f1-score   support

           1       0.80      0.57      0.67      4529
           2       0.46      0.58      0.51      6715
           3       0.58      0.59      0.58      7531
           4       0.75      0.38      0.50      1076

   micro avg       0.57      0.57      0.57     19851
   macro avg       0.65      0.53      0.57     19851
weighted avg       0.60      0.57      0.57     19851

         countF                        GroundTruth percentage
Predict       1       2       3      4                       
pat                                                          
1009       46.0  1110.0   136.0    NaN         1.0   3.560372
1025     1456.0   185.0     7.0    NaN         1.0  88.349515
1028     1082.0   211.0   296.0    NaN         1.0  68.093140
2013       80.0  1334.0  1126.0   25.0         2.0  52.007797
2016       32.0   959.0   462.0   15.0         2.0  65.326975
2019      119.0  1089.0   562.0    NaN         2.0  61.525424
2040   